In [4]:
import pandas as pd
import pickle

In [5]:
with open('data/movimentacao_contas.pkl', 'rb') as f:
    df_movimento = pickle.load(f)
df_movimento

,data_cadastro,tipo,categoria_pagamento,subcategoria_pagamento,pago,conta,descricao,observacoes,id_transferencia,paciente,medico,forma_pagamento,convenio,servicos,origem
0,2025-01-01,SAIDA,SALDO INICIAL,AJUSTE,3178.08,SANTANDER,SALDO INICIAL,Definição de saldo inicial para a conta SANTAN...,None,None,None,None,None,None,SISTEMA
1,2025-01-01,ENTRADA,SALDO INICIAL,AJUSTE,0.33,BANESE,SALDO INICIAL,Definição de saldo inicial para a conta BANESE.,None,None,None,None,None,None,SISTEMA
2,2025-01-01,ENTRADA,SALDO INICIAL,AJUSTE,11.36,C6,SALDO INICIAL,Definição de saldo inicial para a conta C6.,None,None,None,None,None,None,SISTEMA
3,2025-01-01,SAIDA,SALDO INICIAL,AJUSTE,10091.82,CONTA PIX,SALDO INICIAL,Definição de saldo inicial para a conta CONTA ...,None,None,None,None,None,None,SISTEMA
4,2025-01-01,ENTRADA,SALDO INICIAL,AJUSTE,260.86,DINHEIRO,SALDO INICIAL,Definição de saldo inicial para a conta DINHEIRO.,None,None,None,None,None,None,SISTEMA


In [33]:
with open('data/movimento_clinica.pkl', 'rb') as f:
    df_clinica = pickle.load(f)
with open('data/movimento_laboratorio.pkl', 'rb') as f:
    df_laboratorio = pickle.load(f)
with open('data/convenios_detalhados.pkl', 'rb') as f:
    df_convenios = pickle.load(f)

In [78]:
import os
from datetime import datetime
caminho_pendentes = 'data/recebimentos_pendentes.pkl'
novos_pendentes = []

# Processa pendências da Clínica
if df_clinica is not None and not df_clinica.empty and 'a_pagar' in df_clinica.columns:
    df_pend_clinica = df_clinica[df_clinica['a_pagar'] > 0].copy()
    if not df_pend_clinica.empty:
        df_pend_clinica.rename(columns={'a_pagar': 'valor_pendente', 'data_cadastro': 'data_operacao'}, inplace=True)
        df_pend_clinica['origem_recebimento'] = df_pend_clinica['convenio']
        novos_pendentes.append(df_pend_clinica[['data_operacao', 'paciente', 'origem_recebimento', 'valor_pendente','origem']])

# Processa pendências do Laboratório
if df_laboratorio is not None and not df_laboratorio.empty and 'a_pagar' in df_laboratorio.columns:
    df_pend_lab = df_laboratorio[df_laboratorio['a_pagar'] > 0].copy()
    if not df_pend_lab.empty:
        df_pend_lab.rename(columns={'a_pagar': 'valor_pendente', 'data_cadastro': 'data_operacao'}, inplace=True)
        df_pend_lab['origem_recebimento'] = df_pend_lab['convenio']
        novos_pendentes.append(df_pend_lab[['data_operacao', 'paciente', 'origem_recebimento', 'valor_pendente','origem']])

# Processa pendências de Convênios
if df_convenios is not None and not df_convenios.empty:
    df_pend_conv = df_convenios.copy()
    df_pend_conv.rename(columns={'valor': 'valor_pendente', 'data_cadastro': 'data_operacao', 'convenio': 'origem_recebimento'}, inplace=True)
    novos_pendentes.append(df_pend_conv[['data_operacao', 'paciente', 'origem_recebimento', 'valor_pendente', 'origem']])

# Carrega os dados pendentes existentes
if os.path.exists(caminho_pendentes):
    df_pendentes_existente = pd.read_pickle(caminho_pendentes)
else:
    # Se o arquivo não existe, cria um DataFrame vazio para garantir a concatenação
    df_pendentes_existente = pd.DataFrame(columns=['data_operacao', 'paciente', 'origem_recebimento', 'valor_pendente', 'origem', 'id_pendencia'])

# Se houver novas pendências, processa e adiciona
if novos_pendentes:
    df_novos_pendentes = pd.concat(novos_pendentes, ignore_index=True)
    df_novos_pendentes['id_pendencia'] = [f"pend_{int(datetime.now().timestamp())}_{i}" for i in range(len(df_novos_pendentes))]
    
    # Concatena os dados existentes com os novos
    df_pendentes_atualizado = pd.concat([df_pendentes_existente, df_novos_pendentes], ignore_index=True)
else:
    # Se não houver novas pendências, o arquivo atualizado é o mesmo que o existente
    df_pendentes_atualizado = df_pendentes_existente

# # Remove duplicatas para garantir consistência e salva o arquivo final
df_pendentes_atualizado.drop_duplicates(subset=['data_operacao', 'paciente', 'valor_pendente', 'origem_recebimento', 'id_pendencia'], keep='last', inplace=True)
df_pendentes_atualizado.to_pickle(caminho_pendentes)

C:\Users\leopa\AppData\Local\Temp\ipykernel_9332\1610445931.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_pendentes_atualizado = pd.concat([df_pendentes_existente, df_novos_pendentes], ignore_index=True)


In [79]:
df_pendentes_atualizado

,data_operacao,paciente,origem_recebimento,valor_pendente,origem,id_pendencia
0,2025-08-01,NAGILSON DE JESUS SANTOS,IPES,21.18,clinica,pend_1754425212_0
1,2025-08-01,JOSÉ BISPO DOS SANTOS ALVES,IPES,211.32,clinica,pend_1754425212_1
2,2025-08-01,HELOISA SANTOS OLIVEIRA,IPES,114.40,clinica,pend_1754425212_2
3,2025-08-04,ZILMARIO VIEIRA SANTANA,IPES,21.18,clinica,pend_1754425212_3
4,2025-08-01,CLODOMIR BARRETO COSTA,IPES,77.69,clinica,pend_1754425212_4
...,...,...,...,...,...,...
6889,2025-08-04,JOSÉ ANTONIO DOS SANTOS,SUS,1.85,convenios,pend_1754425212_6889
6890,2025-08-04,JOSÉ ANTONIO DOS SANTOS,SUS,4.11,convenios,pend_1754425212_6890
6891,2025-08-04,JOSÉ ANTONIO DOS SANTOS,SUS,1.85,convenios,pend_1754425212_6891
6892,2025-08-04,JOSÉ ANTONIO DOS SANTOS,PARTICULAR,40.00,convenios,pend_1754425212_6892


In [ ]:

# Remove duplicatas para garantir consistência e salva o arquivo final
df_pendentes_atualizado.drop_duplicates(subset=['data_operacao', 'paciente', 'valor_pendente', 'origem_recebimento'], keep='last', inplace=True)
df_pendentes_atualizado.to_pickle(caminho_pendentes)

In [27]:
df = pd.read_html('relMovimentoDiarioDetalhado_04_08_2025 (1).xls')[0].iloc[5:-6]
df.columns = df.columns.get_level_values(-1)
df

,0,1,2,3,4,5,6,7,8,9,10,11
5,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,Relatorio Movimento Diário Detalhado,NaN
6,Cadastro,Unidade,Código,Paciente,Convenio,Total R$,Desc. R$,Acres. R$,Pago R$,Form. Pag.,Atendente,A Pag. R$
7,01/08/2025,RSS,0008036,BEATRIZ SANTOS BENIGNO,PARTICULAR,"R$ 12,00","R$ 0,00","R$ 0,00","R$ 12,00",Dinheiro,CINTIA REGINA LIMA SOUZA,"R$ 0,00"
8,01/08/2025,RSS,0008038,GEANE MARY SANTOS SANTANA,PARTICULAR,"R$ 13,20","R$ 0,20","R$ 0,00","R$ 13,00",Dinheiro,CINTIA REGINA LIMA SOUZA,"R$ 0,00"
9,01/08/2025,RSS,0008040,MARIA LUZIA DA CONCEIÇAO SANTOS,PARTICULAR,"R$ 146,20","R$ 0,00","R$ 0,00","R$ 146,20",Cartão de crédito,CINTIA REGINA LIMA SOUZA,"R$ 0,00"
10,01/08/2025,RSS,0008045,JOEZIO SANTOS RESENDE,PARTICULAR,"R$ 102,00","R$ 7,00","R$ 0,00","R$ 95,00",Dinheiro,CINTIA REGINA LIMA SOUZA,"R$ 0,00"
11,01/08/2025,RSS,0008046,JULIANA DOS SANTOS RIBEIRO JESUS,PARTICULAR,"R$ 31,20","R$ 3,20","R$ 0,00","R$ 28,00",PIX JULIO,LUANNY,"R$ 0,00"
12,01/08/2025,RSS,0008047,ILDE COSME DOS SANTOS OLIVEIRA,PARTICULAR,"R$ 255,10","R$ 12,76","R$ 0,00","R$ 242,34",PIX JULIO,VIVIAN MARIÁ SILVA SANTOS,"R$ 0,00"
13,01/08/2025,RSS,0008048,TAMIRES SANTOS FAUSTINO,LABORATORIO ASO,"R$ 14,00","R$ 0,00","R$ 0,00","R$ 14,00",Dinheiro,CINTIA REGINA LIMA SOUZA,"R$ 0,00"
14,01/08/2025,RSS,0008049,UERICA MACEDO SANTIAGO,PARTICULAR,"R$ 15,40","R$ 0,77","R$ 0,00","R$ 14,63",PIX JULIO,VIVIAN MARIÁ SILVA SANTOS,"R$ 0,00"


In [8]:
df = pd.read_html('relMovimentoDiarioDetalhado_15_07_2025.xls', header=6)[0].iloc[:-5]
df

,Cadastro,Unidade,Código,Paciente,Convenio,Total R$,Desc. R$,Acres. R$,Pago R$,Form. Pag.,Atendente,A Pag. R$
0,01/07/2025,RSS,0007506,ELIZA FERREIRA DE OLIVEIRA,PARTICULAR,"R$ 37,60","R$ 1,60","R$ 0,00","R$ 27,00",Dinheiro,LUANNY,"R$ 0,00"
1,01/07/2025,RSS,0007506,ELIZA FERREIRA DE OLIVEIRA,PARTICULAR,"R$ 37,60","R$ 1,60","R$ 0,00","R$ 9,00",Dinheiro,LUANNY,"R$ 0,00"
2,01/07/2025,RSS,0007509,JOSE RAIMUNDO DOS SANTOS,PARTICULAR,"R$ 243,20","R$ 13,20","R$ 0,00","R$ 230,00",PIX JULIO,CINTIA REGINA LIMA SOUZA,"R$ 0,00"
3,01/07/2025,RSS,0007511,RITA DE CASSIA SILVA FONSECA,PARTICULAR,"R$ 18,70","R$ 0,94","R$ 0,00","R$ 17,76",PIX JULIO,VIVIAN MARIÁ SILVA SANTOS,"R$ 0,00"
4,02/07/2025,RSS,0007514,ADRIANA ALEMÃO SANTOS NOBRE,PARTICULAR,"R$ 13,20","R$ 0,00","R$ 0,80","R$ 14,00",PIX JULIO,CINTIA REGINA LIMA SOUZA,"R$ 0,00"
...,...,...,...,...,...,...,...,...,...,...,...,...
193,15/07/2025,RSS,0007688,JOSEFA LEITE SANTOS,SUS,"R$ 99,37","R$ 0,00","R$ 0,00","R$ 0,00",NaN,CINTIA REGINA LIMA SOUZA,"R$ 99,37"
194,15/07/2025,RSS,0007690,CLEOMISON ROSA SANTOS,SUS,"R$ 42,78","R$ 0,00","R$ 0,00","R$ 0,00",NaN,CINTIA REGINA LIMA SOUZA,"R$ 42,78"
195,15/07/2025,RSS,0007692,MARIA AUGUSTA ALVES DE OLIVEIRA,IPES,"R$ 336,23","R$ 0,00","R$ 0,00","R$ 0,00",NaN,LUANNY,"R$ 336,23"
196,15/07/2025,RSS,0007695,MARIA ELISA LIMA CALANZANI,IPES,"R$ 22,38","R$ 0,00","R$ 0,00","R$ 0,00",NaN,CINTIA REGINA LIMA SOUZA,"R$ 22,38"


In [10]:
df = pd.read_excel('relConvDetalhadoPorLinha_15_07_2025.xlsx', header=5)
df

,Data,Documento,Empresa,Código,Exame,Descrição,Código Exame,Paciente,Endereço,Valor,Unidade,Local,Convênio,Matricula
0,01/07/2025,NaN,NaN,7502,URE,URÉIA,020201069-4,PAULO CESAR SANTOS DE OLIVEIRA,"povoado saco da raiz,65","1,85",REDE SANTA SAÚDE,NaN,SUS,NaN
1,01/07/2025,NaN,NaN,7502,TRI,TRIGLICERIDEOS,020201067-8,PAULO CESAR SANTOS DE OLIVEIRA,"povoado saco da raiz,65","3,51",REDE SANTA SAÚDE,NaN,SUS,NaN
2,01/07/2025,NaN,NaN,7502,TGO,TRANSAMINASE OXALÁCETICA (TGO),020201064-3,PAULO CESAR SANTOS DE OLIVEIRA,"povoado saco da raiz,65","2,01",REDE SANTA SAÚDE,NaN,SUS,NaN
3,01/07/2025,NaN,NaN,7502,TGP,TRANSAMINASE PIRUVICA (TGP),020201065-1,PAULO CESAR SANTOS DE OLIVEIRA,"povoado saco da raiz,65","2,01",REDE SANTA SAÚDE,NaN,SUS,NaN
4,01/07/2025,NaN,NaN,7502,URI,URINA TIPO I,020205001-7,PAULO CESAR SANTOS DE OLIVEIRA,"povoado saco da raiz,65","3,70",REDE SANTA SAÚDE,NaN,SUS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2267,15/07/2025,NaN,NaN,7695,PPF3,PARASITOLÓGICO DE FEZES - 3º AMOSTRA,40303110,MARIA ELISA LIMA CALANZANI,RUA AQUIDABA 222,"7,46",REDE SANTA SAÚDE,NaN,IPES,NaN
2268,15/07/2025,NaN,NaN,7696,PPF2,PARASITOLÓGICO DE FEZES - 2º AMOSTRA,40303110,FERNANDO LIMA CALANZANI,RUA AQUIDABA222,"7,46",REDE SANTA SAÚDE,NaN,IPES,NaN
2269,15/07/2025,NaN,NaN,7696,PPF1,PARASITOLÓGICO DE FEZES - 1º AMOSTRA,40303110,FERNANDO LIMA CALANZANI,RUA AQUIDABA222,"7,46",REDE SANTA SAÚDE,NaN,IPES,NaN
2270,15/07/2025,NaN,NaN,7696,PPF3,PARASITOLÓGICO DE FEZES - 3º AMOSTRA,40303110,FERNANDO LIMA CALANZANI,RUA AQUIDABA222,"7,46",REDE SANTA SAÚDE,NaN,IPES,NaN
